1. Предлагается провести обучение модели Линейной регрессии и CatBoost.
2. red_channel_average,green_channel_average,blue_channel_average


In [1]:
import os
import seaborn as sns
import numpy as np
import pandas as pd
from skimage import io
from matplotlib import pyplot as plt
import cv2
from skimage.feature import hog
from PIL import Image, ImageChops

def calculate_channel_average(img, channel):
    channel_dict = {"R": 0, "G": 1, "B": 2}
    channel_idx = channel_dict[channel]
    channel_intensities = np.array([row[:, channel_idx] for row in img]).flatten()
    return np.mean(channel_intensities)
    
def data_dictionary(part="train"):
    path_train = f"/home/roman/Documents/hse/skin_problems/{part}/"
    train_data_categories = os.listdir(path_train)
    list_train = train_data_categories
    train_dictionary = {"image_path": [],
                        "target": [], 
                        "Label": [], 
                        "Image": [],
                        'red_channel_intensity':[],
                        'blue_channel_intensity':[],
                        'green_channel_intensity':[],
                        'red_channel_average':[],
                        'blue_channel_average':[],
                        'green_channel_average':[]}
    train_dictionary = pd.DataFrame(train_dictionary)
    k = 0
    for i in list_train:
        path_train_item = path_train + "/" + i
        image_list_train = os.listdir(path_train_item)
        for j in image_list_train:
            img_path_train = path_train_item + "/" + j
            image = io.imread(img_path_train)
            red_channel_intensity = calculate_channel_average(image,"R")
            blue_channel_intensity = calculate_channel_average(image,"B") 
            green_channel_intensity = calculate_channel_average(image,"G")
            new_row = {
                       'image_path': img_path_train, 
                       'target':  k,
                       'Label': i,
                       'red_channel_intensity': red_channel_intensity,
                       'blue_channel_intensity': blue_channel_intensity,
                       'green_channel_intensity': green_channel_intensity,
                       'Image': image
                      }
            train_dictionary.loc[len(train_dictionary)] = new_row
        k += 1
        train_dictionary.loc[train_dictionary['Label'] == i, 'red_channel_average'] = train_dictionary['red_channel_intensity'].agg("mean")
        train_dictionary.loc[train_dictionary['Label'] == i, 'blue_channel_average'] = train_dictionary['blue_channel_intensity'].agg("mean")
        train_dictionary.loc[train_dictionary['Label'] == i, 'green_channel_average'] = train_dictionary['green_channel_intensity'].agg("mean")
    return train_dictionary



dataframe_train = data_dictionary(part = "train")
dataframe_test = data_dictionary(part = "test")

print(dataframe_train)


    


/home/roman/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:950: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:950: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If

                                              image_path  target  \
0      /home/roman/Documents/hse/skin_problems/train/...       0   
1      /home/roman/Documents/hse/skin_problems/train/...       0   
2      /home/roman/Documents/hse/skin_problems/train/...       0   
3      /home/roman/Documents/hse/skin_problems/train/...       0   
4      /home/roman/Documents/hse/skin_problems/train/...       0   
...                                                  ...     ...   
15552  /home/roman/Documents/hse/skin_problems/train/...      22   
15553  /home/roman/Documents/hse/skin_problems/train/...      22   
15554  /home/roman/Documents/hse/skin_problems/train/...      22   
15555  /home/roman/Documents/hse/skin_problems/train/...      22   
15556  /home/roman/Documents/hse/skin_problems/train/...      22   

                                                   Label  \
0           Light Diseases and Disorders of Pigmentation   
1           Light Diseases and Disorders of Pigmentation   
2  

In [2]:
pip install catboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score
print(f" missing values in test dataset : {dataframe_test.isnull().sum()}")
print(f" missing values in train dataset : {dataframe_train.isnull().sum()}")

df_train = dataframe_train.copy()
df_test = dataframe_test.copy()
y_train = df_train['target']
df_train.drop('Label', axis=1, inplace=True)
df_train.drop('Image', axis=1, inplace=True)
df_train.drop('image_path', axis=1, inplace=True)
df_train.drop('target', axis=1, inplace=True)

df_train.drop('red_channel_intensity', axis=1, inplace=True)
df_train.drop('blue_channel_intensity', axis=1, inplace=True)
df_train.drop('green_channel_intensity', axis=1, inplace=True)

X_train = df_train
y_test = df_test['target']
df_test.drop('Label', axis=1, inplace=True)
df_test.drop('Image', axis=1, inplace=True)
df_test.drop('image_path', axis=1, inplace=True)
df_test.drop('target', axis=1, inplace=True)

df_test.drop('red_channel_intensity', axis=1, inplace=True)
df_test.drop('blue_channel_intensity', axis=1, inplace=True)
df_test.drop('green_channel_intensity', axis=1, inplace=True)


X_test = df_test

# модель
model = LogisticRegression(solver='lbfgs', max_iter=10000, random_state=42)
model.fit(X_train, y_train)
y_pred_test = model.predict(X_test)

# результаты 
print(f'LogReg accuracy: {accuracy_score(y_test, y_pred_test):.4f}')




image_path                 0
target                     0
Label                      0
Image                      0
red_channel_intensity      0
blue_channel_intensity     0
green_channel_intensity    0
red_channel_average        0
blue_channel_average       0
green_channel_average      0
dtype: int64
image_path                 0
target                     0
Label                      0
Image                      0
red_channel_intensity      0
blue_channel_intensity     0
green_channel_intensity    0
red_channel_average        0
blue_channel_average       0
green_channel_average      0
dtype: int64
LogReg accuracy: 0.4658
Cat boost Mean Squared Error = 17.282344648994673


In [10]:
from catboost import CatBoostClassifier
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# Создание экземпляра модели
cat = CatBoostRegressor()

# Обучение модели
cat.fit(X_train, y_train, verbose=False)

# Применение модели на тестовых данных
preds = cat.predict(X_test)

metrics = {}
metrics['mape'] = mean_absolute_percentage_error(y_test, preds)
metrics['mae'] = mean_absolute_error(y_test, preds)

print(metrics)

Learning rate set to 0.063169
0:	learn: 6.7545584	total: 728us	remaining: 727ms
1:	learn: 6.3422335	total: 1.4ms	remaining: 700ms
2:	learn: 5.9568815	total: 2.22ms	remaining: 737ms
3:	learn: 5.6027244	total: 2.83ms	remaining: 705ms
4:	learn: 5.2650530	total: 3.4ms	remaining: 677ms
5:	learn: 4.9525036	total: 4.05ms	remaining: 671ms
6:	learn: 4.6598081	total: 4.61ms	remaining: 654ms
7:	learn: 4.3821695	total: 5.17ms	remaining: 641ms
8:	learn: 4.1179445	total: 5.77ms	remaining: 635ms
9:	learn: 3.8763466	total: 6.35ms	remaining: 629ms
10:	learn: 3.6448445	total: 6.98ms	remaining: 628ms
11:	learn: 3.4304635	total: 7.56ms	remaining: 623ms
12:	learn: 3.2314080	total: 8.12ms	remaining: 617ms
13:	learn: 3.0423716	total: 8.69ms	remaining: 612ms
14:	learn: 2.8650591	total: 9.29ms	remaining: 610ms
15:	learn: 2.6975432	total: 10ms	remaining: 617ms
16:	learn: 2.5487237	total: 10.6ms	remaining: 614ms
17:	learn: 2.4038014	total: 11.2ms	remaining: 612ms
18:	learn: 2.2658683	total: 11.8ms	remaining: 609